In [1]:
# Initialisation
import os
import requests
import json
import pandas as pd
import urllib.parse as url
from dotenv import load_dotenv

In [2]:
# Initialised
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension()

In [3]:
load_dotenv()
api_key = os.getenv("DOMAIN_API_KEY")

In [13]:
title = "All In One Portfolio Summary and Analysis"

In [108]:
# REQUEST CALL DOMAIN.COM.AU: This api call will, given a string, finds the property address that closest matches the string
# and returns the address in full as a string, each address components in its own string, and a unique ID for the given address.
# We will return all this data in a df
def address_suggest(address):
    if not address:
        return
    address_url = url.quote(address)
    request_url = f"https://api.domain.com.au/v1/properties/_suggest?terms={address_url}&pageSize=1&api_key={api_key}"
    get_request = requests.get(request_url)

    # Data formatting
    data = get_request.json()
    # Extract data
    full_address = data[0]["address"]
    address_data = data[0]["addressComponents"]
    location_id = data[0]["id"]
    
    
    # Address_data is already a dict so we need to combine everything with update
    all_data = {
        "full_address": full_address,
        "id": location_id
    }
    all_data.update(address_data)
    # Put data into df
    # Df will have a single row. All data in their own columns with their own names
    address_df = pd.DataFrame([all_data])
    return address_df

In [135]:
# Address locators: this request will return a unique identifier for the following address levels:
# Address, street, suburb, postcode

def address_locators(address_df, level, building_type):

    if building_type == "Unit":
        unit = address_df.iloc[0]["unitNumber"]
    else:
        unit = None
    street_num = address_df.iloc[0]["streetNumber"]
    street_name = address_df.iloc[0]["streetName"]
    street_type = address_df.iloc[0]["streetType"]
    suburb = address_df.iloc[0]["suburb"]
    post_code = address_df.iloc[0]["postCode"]
    state = address_df.iloc[0]["state"]
    
    # Check if building is actually a unit
    if unit_flag:
        request_url = f"https://api.domain.com.au/v1/addressLocators?searchLevel={level}&unitNumber={unit}&streetNumber={street_num}&streetName={street_name}&streetType={street_type}&suburb={suburb}&state={state}&postcode={post_code}&api_key={api_key}"

    else:
        request_url = f"https://api.domain.com.au/v1/addressLocators?searchLevel={level}&streetNumber={street_num}&streetName={street_name}&streetType={street_type}&suburb={suburb}&state={state}&postcode={post_code}&api_key={api_key}"

    get_request = requests.get(request_url)

    # Data formatting
    data = get_request.json()
    print(data)
    combined_id_data = {}
    for loc_id in data[0]["ids"]:
        combined_id_data.update({loc_id["level"]+" id": loc_id["id"]})
    
    id_df = pd.DataFrame([combined_id_data])
    
    return id_df

In [141]:
# Properties: Using a given property id provided by _suggest, this request gives an info dump on the characteristics of the property
# For now just returning all data since I do not know exactly what data is required
def properties(prop_id):
    request_url = f"https://api.domain.com.au/v1/properties/{prop_id}?api_key={api_key}"
    get_request = requests.get(request_url)
    data = get_request.json()
    return data

In [142]:
# Top level of property section will do api calls on _suggest and address-locators, determine if property is a unit
# and will produce an info dump on the selected property
def property_top_level(address):
    if not address:
        return
    address_df = address_suggest(address)
    address_df.head()
    full_address = address_df.iloc[0]["full_address"]
    address_string = f"Showing data for the following address:  <br /> {full_address}"
    
    # Create a flag here on top level to indicate if property is a unit
    # unit number wont be a column in address_df if the property isn't a unit to begin with
    if address_df.iloc[0]["unitNumber"]:
        building_type = "Unit"
    else:
        building_type = "House"
       
    id_df = address_locators(address_df, "address", building_type)
    
    property_data = properties(address_df.iloc[0]["id"])
    
    bedrooms = property_data["bedrooms"]
    
    info_string
    
    return pn.Pane(info_string, width = 1000)

In [138]:
def filler_stock_function(stocks):
    print(stocks)

In [139]:
# Put everything together into this dashboard and serve it
property_opening_message = """Please enter your full address  <br />
This program will find the address with the closest match to the address you provide it"""

property_opening_column = pn.Column(property_opening_message,
                                   interact(property_top_level, 
                                            address=widgets.TextInput(name="Your Address", placeholder="Please enter your address here")),
                                   width=600)

stock_opening_message = """Please upload a .csv file with your stock symbol, purchase date, and purchase amount  <br />
Please use the following column names in your csv: symbol, date, amount"""

stock_analysis_opening_column = pn.Column(stock_opening_message, 
                                          interact(filler_stock_function, stocks=widgets.FileInput(accept=".csv")),
                                         width = 600)

tabs = pn.Tabs(
    ("Property Analysis", 
     property_opening_column),
    ("Stock Portfolio Analysis",
    stock_analysis_opening_column)
)   
dashboard = pn.Column(title, tabs)
dashboard.servable()

None


Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column(width=600)
            [0] Markdown(str)
            [1] Column
                [0] Column
                    [0] TextInput(name='Your Address', placeholder='Please enter y...)
                [1] Row
                    [0] Str(None, name='interactive01475')
        [1] Column(width=600)
            [0] Markdown(str)
            [1] Column
                [0] Column
                    [0] FileInput(accept='.csv')
                [1] Row
                    [0] Str(None, name='interactive01485')

[{'types': ['Address'], 'addressComponents': [{'component': 'StreetNumber', 'shortName': '10'}, {'component': 'StreetName', 'shortName': 'Crower'}, {'component': 'StreetType', 'shortName': 'Rd'}, {'component': 'Suburb', 'shortName': 'Avenue Range'}, {'component': 'Postcode', 'shortName': '5273'}, {'component': 'State', 'shortName': 'SA'}], 'ids': [{'level': 'Address', 'id': 24501525}, {'level': 'Street', 'id': 436575}, {'level': 'Suburb', 'id': 545}, {'level': 'Postcode', 'id': 3505}]}]


NameError: name 'info_string' is not defined